In [ ]:
!pip install torch-geometric

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


In [ ]:
!nvcc --version

if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

print("Using {}.".format(device_name))

In [ ]:
#data

import numpy as np
import torch
import random
from torch_geometric.data import Data, DataLoader

#Import data function
def get_data(filepath):
    data = np.loadtxt(filepath)
    x = data[:, :20]
    z = data[:, 20:40]
    m = data[:, 40:60]

    dataX = []
    for i in range(x.shape[0]):
        dataXs = np.column_stack((np.arange(20), x[i], z[i]))
        dataX.append(dataXs)

    dataY = m
    zipped = list(zip(dataX, dataY))
    np.random.shuffle(zipped)
    dataX, dataY = zip(*zipped)

    dataX = np.array(dataX)
    dataY = np.array(dataY)

    dataEdgeIndex = np.column_stack((np.arange(20)[:-1], np.arange(20)[1:]))
    dataEdgeIndex = np.vstack((dataEdgeIndex, dataEdgeIndex[:, ::-1])).T

    dataset = [Data(x=torch.from_numpy(x).float(), edge_index=torch.from_numpy(dataEdgeIndex).long(), y=torch.from_numpy(y).float()) for x, y in zip(dataX, dataY)]

    return dataset


#Getting data
dataset = get_data("/content/drive/MyDrive/datasets/data1.txt")
print(len(dataset))
#Splitting dataset
datasetRun = dataset[:int(len(dataset)*0.90)]
datasetTest = dataset[int(len(dataset)*0.90):]

train_loader = datasetRun[:int(len(datasetRun)*0.75)]
test_loader = datasetRun[int(len(datasetRun)*0.75):]



In [ ]:
#Data Checks

print(len(train_loader))
print(len(test_loader))

# one data object from train_loader:
graph = train_loader[0]
print(graph.x.shape)
print(graph.edge_index.shape)
print(graph.y.shape)

In [ ]:
# ArchNN

class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True, l1_lambda=0.001):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16) 
        #self.bn0 = nn.BatchNorm1d(16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        #self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 256)
        #self.bn1 = nn.BatchNorm1d(256) 
        self.fc2 = nn.Linear(256, 1)

        self.reset_parameters()
        self.l1_lambda = l1_lambda

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        #self.conv4.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.fc0(x))
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        #x = F.elu(self.conv4(x, edge_index))
        x = F.elu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x

    """def compute_loss(self, outputs, targets):
      # Define the loss function
      mse_loss = nn.MSELoss()
      l1_loss = torch.tensor(0.0, requires_grad=True).to(outputs.device)  # initialize L1 loss to 0
      for name, param in self.named_parameters():
          if 'weight' in name:  # apply L1 regularization only to weight parameters
              l1_loss += torch.norm(param, p=1)  # compute element-wise L1 norm
      loss = mse_loss(outputs, targets) + self.l1_lambda * l1_loss  # add L1 regularization term to the loss
      return loss"""

In [ ]:

import time
import torch
import torch.nn.functional as F


def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'])
    print(message)


def run(model, train_loader, test_loader, num_nodes, epochs, optimizer, device):

    for epoch in range(1, epochs + 1):
        #random.shuffle(train_loader) #added this this see if it helps, tihihi
        #random.shuffle(test_loader)  #added this this see if it helps, tihihi
        t = time.time()
        train_loss = train(model, train_loader, optimizer, device)
        t_duration = time.time() - t
        test_loss = test(model, test_loader, num_nodes, device)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)


def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        #log_probs = F.log_softmax(output, dim=1).double()
        #print(log_probs.dtype)
        #print(data.y.dtype)
        loss = F.mse_loss(output, data.y.to(device))

        #for i in range(data.num_nodes):
        #    loss = F.mse_loss(output[i], data.y[i].to(device))
        #    loss.backward(retain_graph=True)

        #loss = model.compute_loss(output, data.y)  # compute loss with L1 regularization
        #loss = F.nll_loss(log_probs, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data.to(device))
            total_loss += F.mse_loss(out, data.y.to(device)).item()
            #pred = out.max(1)[1]
            #correct += pred.eq(data.y).sum().item()
            #n_graphs += data.num_graphs
    return total_loss / len(test_loader)


In [ ]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model = ArchNN(num_features, num_nodes, heads=8).to(device_name)

optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0005)


run(model, train_loader, test_loader, num_nodes, 50, optimizer, device_name)

In [ ]:
#More epochs per favoure?
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

run(model, train_loader, test_loader, num_nodes, 25, optimizer, device_name)

In [ ]:
#Test for new dataline

def testData(line):
    real = datasetTest[line].y
    out = model(datasetTest[line].to(device_name))
    Bool = ''
    print('{:<10}{:<10}{:<10}{:<10}'.format('real', 'out', '%error', 'Sign'))
    for i in range(len(out)):
        error = ((out-real[line])/out)*100
        if torch.sign(real[i]) == torch.sign(datasetTest[line].y[i]):
            Bool = 'True'
        else:
            Bool = 'False'
        real_formatted = '{:<10.2f}'.format(round(real[i].item(),2))
        out_formatted = '{:<10.2f}'.format(round(out[i].item(),2))
        error_formatted = '{:<10.2f}'.format(round(error[i].item(),2))
        list_formatted = [real_formatted, out_formatted, error_formatted, Bool]
        print('{:<10}{:<10}{:<10}{:<10}'.format(*list_formatted))

#Test
testData(0)


In [ ]:
#Test for new dataset
def testFinal(model, indata, device):
    model.eval()
    #correct = 0
   # total_loss = 0
   # n_graphs = 0
    errorAVG = 0
    with torch.no_grad():
        for id, data in enumerate(indata):
          out = model(data.to(device))
          real = data.y.to(device)
          errorList = []
          error = 0
          for i in range(len(out)):
            errorList.append(round(((out[i].item() - real[i].item() ) / out[i].item())*100, 2))
            error += abs(((out[i].item() - real[i].item() ) / out[i].item())*100)

          errorAVG += round(error/len(out),2)
          print(errorList)
          print('Avarage error: '+ str(round(error/len(out),2))+'%')
          print('Mean Square: '+ str(round(F.mse_loss(out, data.y.to(device)).item(),2))+'\n')
    
    print('The avarage error is ' + str(errorAVG/len(indata))+'%')


#Load test data set, if this is not done from beginning!
#datasetTest = get_data("/content/drive/MyDrive/datasets/data1test.txt")

#Run testFinal
testFinal(model, datasetTest,  device_name)


In [ ]:
#test runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

modelTest = ArchNN(num_features, num_nodes, heads=8).to(device_name)

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.005)


run(modelTest, train_loader, test_loader, num_nodes, 5, optimizer, device_name)